In [1]:
from p_plp.db import *
from p_plp.feature_engineering import *
from p_plp.cohorts import *
from p_plp.modeling import *

In [2]:
engine = get_engine()

In [3]:

df_person = get_cdm_table(engine, CdmTable.DRUG_EXPOSURE)
df_person.head()


,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,1,1,1710617,2005-09-22,2005-09-22,2006-01-29,2006-01-29,2006-01-29,32838,None,...,None,0,0,236,11,1000011,199663,1710617,None,None
1,2,1,19122186,2005-09-22,2005-09-22,2006-01-29,2006-01-29,2006-01-29,32838,None,...,None,0,0,236,11,1000011,597730,19122186,None,None
2,3,1,19122186,2008-10-09,2008-10-09,2008-10-09,2008-10-09,None,32838,None,...,None,0,0,236,21,1000021,597730,19122186,None,None
3,4,1,1703093,2008-10-09,2008-10-09,2008-10-09,2008-10-09,None,32838,None,...,None,0,0,236,21,1000021,476556,1703093,None,None
4,5,1,40224805,1998-07-30,1998-07-30,1999-07-25,1999-07-25,1999-07-25,32838,None,...,None,0,0,62,42,1000042,1000126,40224805,None,None


In [4]:
list_observed_conditions(engine).head()

,concept_id,concept_name,vocabulary_id,n_occurrences
0,4281516,Gingivitis,SNOMED,264
1,133228,Dental caries,SNOMED,115
2,40481087,Viral sinusitis,SNOMED,108
3,4112343,Acute viral pharyngitis,SNOMED,73
4,260139,Acute bronchitis,SNOMED,66


In [5]:
list_observed_outcomes(engine).head()

,concept_id,concept_name,vocabulary_id,n_occurrences
0,4281516,Gingivitis,SNOMED,264
1,133228,Dental caries,SNOMED,115
2,40481087,Viral sinusitis,SNOMED,108
3,4112343,Acute viral pharyngitis,SNOMED,73
4,260139,Acute bronchitis,SNOMED,66


In [6]:
generate_target_cohort(engine, 40481087) 
generate_outcome_cohort(engine, 133228)  
generate_labels_time_at_risk(engine)

,subject_id,index_date,tar_start_date,tar_end_date,outcome_flag,outcome_date
0,63,1989-01-15,1989-01-16,1998-04-03,0,1987-02-01
1,117,1989-03-12,1989-03-13,1998-05-29,0,None
2,86,1990-03-03,1990-03-04,1999-05-20,1,1995-09-16
3,66,1997-03-30,1997-03-31,2006-06-16,0,None
4,82,1999-06-02,1999-06-03,2008-08-18,0,None
...,...,...,...,...,...,...
67,11,2024-01-05,2024-01-06,2033-03-23,0,2011-02-26
68,34,2024-01-20,2024-01-21,2033-04-07,0,None
69,20,2024-02-08,2024-02-09,2033-04-26,0,2018-11-24
70,57,2024-12-05,2024-12-06,2034-02-21,1,2025-03-06


In [7]:
build_prior_condition_count_features(engine)
get_prior_condition_count_features(engine).head()

,subject_id,n_prior_conditions
0,1,1
1,2,0
2,4,0
3,5,0
4,6,1


In [8]:
build_demographic_features(engine)
get_demographic_features(engine).head()

,subject_id,age,gender_concept_id
0,1,23,8532
1,2,35,8507
2,4,32,8507
3,5,7,8532
4,6,15,8532


In [9]:
build_model_dataset(engine)
df_model = get_model_dataset(engine)

In [10]:
X, y, num_cols, cat_cols = make_X_y(df_model)
X

,age,gender_concept_id,n_prior_conditions
0,63,8507,4
1,37,8532,1
2,22,8507,0
3,72,8507,0
4,44,8532,1
...,...,...,...
67,30,8507,0
68,48,8532,0
69,24,8507,1
70,14,8532,0


In [12]:
model, X_test, y_test = train_pipeline(df_model)